In [1]:
using Pkg
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

Plots.GRBackend()

In [2]:
using LinearAlgebra
using FFTW
using Printf
using Plots

### read Equil

In [24]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
equil_input = JPEC.Equilibrium.EquilInput(
    "beta_1.00",        # eq_filename
    "efit",          # eq_type
    "boozer",        # jac_type
    0.01,             # psilow
    1.0,             # psihigh
    100,             # mpsi (number of radial grid points)
    128              # mtheta (number of poloidal grid points)
)

# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_input)
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...
--- Julia Equilibrium Setup ---
Equilibrium file: beta_1.00
Type = efit, Jac_type = boozer
----------------------------------------
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separ

### Use spline_eval 
sq!!

In [25]:
# The plasma_eq object contains the final 1D profile spline, `sq`.
# We will evaluate it on a fine grid to get smooth plot lines.
psi_norm_grid = range(0.0, 1.0, length=200)

# spline_eval returns the function values (and derivatives if requested).
# For sq, the quantities are: 1:F, 2:P*mu0, 3:Toroidal Flux, 4:q
f_profiles = JPEC.SplinesMod.spline_eval(plasma_eq.sq, collect(psi_norm_grid))

# Extract each profile into its own variable for clarity
F_profile      = f_profiles[:, 1]
P_profile      = f_profiles[:, 2]
tor_flux_prof  = f_profiles[:, 3]
q_profile      = f_profiles[:, 4];

### calc. Metric & Matrix


In [ ]:
"""
fourfit_metric.jl

Julia port of fourfit_make_metric from fourfit.f
Computes Fourier series of metric tensor components for MHD equilibrium analysis.

This module provides functions to:
1. Compute contravariant basis vectors in flux coordinates
2. Calculate metric tensor components (g11, g22, g33, g23, g31, g12)
3. Fit the results to Fourier-spline representation
"""

module FourfitMetric

using LinearAlgebra
using JPEC

export fourfit_make_metric, MetricData, fourfit_make_matrix, MatrixData, compute_eigenvalues

"""
    MetricData

Structure to hold the computed metric tensor data and related information.
Julia equivalent of the Fortran fspline structure.
"""
mutable struct MetricData
    # Grid parameters (Fortran: mpsi, mtheta, mband)
    mpsi::Int          # Number of psi grid points
    mtheta::Int        # Number of theta grid points  
    mband::Int         # Fourier band width
    
    # Coordinate arrays (Fortran: xs, ys)
    xs::Vector{Float64}        # psi coordinates
    ys::Vector{Float64}        # theta coordinates (normalized to 2π)
    
    # Metric tensor components (Fortran: fs array)
    # Components: g11, g22, g33, g23, g31, g12, jmat, jmat1
    fs::Array{Float64,3}       # (mpsi+1, mtheta+1, 8)
    
    # Fitted Fourier-spline representation (Fortran: fitted spline object)
    fspline::Union{Nothing, Any}  # Will hold the fitted spline
    
    # Metadata (Fortran: name, title arrays)
    name::String
    title::Vector{String}
    xtitle::String
    ytitle::String
    
    function MetricData(mpsi, mtheta, mband)
        xs = zeros(mpsi + 1)
        ys = zeros(mtheta + 1)
        fs = zeros(mpsi + 1, mtheta + 1, 8)
        title = ["g11", "g22", "g33", "g23", "g31", "g12", "jmat", "jmat1"]
        
        new(mpsi, mtheta, mband, xs, ys, fs, nothing, "metric", title, "psi", "theta")
    end
end

"""
    MatrixData

Structure to hold the computed MHD matrix data.
Julia equivalent of the Fortran cspline structures for F, G, K matrices.
"""
mutable struct MatrixData
    # Grid parameters
    mpsi::Int
    mpert::Int          # Number of perturbed modes
    mband::Int          # Bandwidth
    mlow::Int           # Lowest mode number
    mhigh::Int          # Highest mode number
    nn::Int             # Toroidal mode number
    
    # Coordinate arrays
    xs::Vector{Float64}  # psi coordinates
    
    # Matrix data - stored as complex splines
    fmats::Union{Nothing, Any}  # F matrix coefficients
    gmats::Union{Nothing, Any}  # G matrix coefficients  
    kmats::Union{Nothing, Any}  # K matrix coefficients
    
    # For boundary conditions
    amat::Union{Nothing, Matrix{ComplexF64}}
    bmat::Union{Nothing, Matrix{ComplexF64}}
    cmat::Union{Nothing, Matrix{ComplexF64}}
    ipiva::Union{Nothing, Vector{Int}}
    
    function MatrixData(mpsi, mpert, mband, mlow, mhigh, nn)
        xs = zeros(mpsi + 1)
        new(mpsi, mpert, mband, mlow, mhigh, nn, xs, 
            nothing, nothing, nothing, nothing, nothing, nothing, nothing)
    end
end

"""
    setup_metric_calculation(plasma_eq, mpsi, mtheta, mband)

Set up the basic parameters for metric calculation from JPEC equilibrium.
"""


"""
    fourfit_make_metric(rzphi, sq; mpsi=100, mtheta=128, mband=10, fft_flag=true)

Main function to compute metric tensor components and fit them to Fourier series.
This is a Julia port of the Fortran subroutine fourfit_make_metric.

Arguments:
- rzphi: JPEC 2D bicubic spline object containing geometry data  
- sq: JPEC 1D spline object containing equilibrium profiles
- mpsi: Number of radial (psi) grid points (default: 100)
- mtheta: Number of poloidal (theta) grid points (default: 128) 
- mband: Fourier bandwidth parameter (default: 10)
- fft_flag: Use FFT-based fitting if true, otherwise use integral method (default: true)

Returns:
- metric: MetricData object containing computed metric tensor components

Fortran equivalent:
```fortran
SUBROUTINE fourfit_make_metric(rzphi, sq, metric, mpsi, mtheta, mband)
```
"""
function fourfit_make_metric(rzphi, sq; 
                           mpsi::Int=100, 
                           mtheta::Int=128, 
                           mband::Int=10, 
                           fft_flag::Bool=true)
    
    println("🔧 Starting metric tensor calculation...")
    println("   Grid: $(mpsi+1) × $(mtheta+1), mband: $mband")
    
    # Constants (Fortran: twopi, ro)
    twopi = 2π
    ro = 1.0  # Major radius - should be obtained from equilibrium data
    
    # Initialize metric data structure
    # Julia equivalent of: CALL fspline_alloc(metric,mpsi,mtheta,mband,8)
    metric = MetricData(mpsi, mtheta, mband)
    
    # Set up coordinate grids
    # Use rzphi coordinate grids to match Fortran exactly
    # Fortran: metric%xs=rzphi%xs; metric%ys=rzphi%ys*twopi
    metric.xs .= rzphi.xs  # Copy psi coordinates from rzphi
    metric.ys .= rzphi.ys .* twopi  # Copy theta coordinates and multiply by 2π
    
    # Temporary arrays for computation (Fortran: v array)
    v = zeros(3, 3)  # Contravariant basis vectors
    
    println("📊 Computing metric tensor components on grid...")
    
    # Main computation loop over all grid points
    # Fortran equivalent: DO ipsi=0,mpsi; DO itheta=0,mtheta
    for ipsi in 0:mpsi
        # Get psi value from equilibrium spline (Fortran: psifac=sq%xs(ipsi))
        psifac = sq.xs[ipsi+1]  # Julia 1-based indexing
        
        for itheta in 0:mtheta
            # Use rzphi coordinate grids exactly as in Fortran
            # Fortran: CALL bicube_eval(rzphi,rzphi%xs(ipsi),rzphi%ys(itheta),1)
            psi_coord = rzphi.xs[ipsi+1]    # rzphi%xs(ipsi)
            theta_coord = rzphi.ys[itheta+1] # rzphi%ys(itheta)
            
            # Evaluate 2D bicubic spline at grid point
            f, fx, fy = JPEC.SplinesMod.bicube_eval(rzphi, psi_coord, theta_coord, 1)
            
            # Extract key geometric quantities (Fortran variable names preserved)
            # Fortran: theta=rzphi%ys(itheta)
            theta = theta_coord
            rfac = sqrt(f[1])           # √(r²)
            eta = twopi * (theta + f[2]) # 2π*(θ + φ_shift)
            r = ro + rfac * cos(eta)    # Major radius R
            jac = f[4]                  # Jacobian
            jac1 = fx[4]               # ∂J/∂ψ
            
            # Compute contravariant basis vectors
            # Fortran: Direct computation of v array elements
            fx1, fx2, fx3 = fx[1], fx[2], fx[3]
            fy1, fy2, fy3 = fy[1], fy[2], fy[3]
            
            # First component (psi direction)
            v[1,1] = fx1 / (2 * rfac * jac)
            v[1,2] = fx2 * twopi * rfac / jac  
            v[1,3] = fx3 * r / jac
            
            # Second component (theta direction)
            v[2,1] = fy1 / (2 * rfac * jac)
            v[2,2] = (1 + fy2) * twopi * rfac / jac
            v[2,3] = fy3 * r / jac
            
            # Third component (phi direction)
            v[3,1] = 0.0
            v[3,2] = 0.0  
            v[3,3] = twopi * r / jac
            
            # Compute metric tensor components (Fortran names preserved)
            g11 = sum(v[1,:].^2) * jac          # g11 = sum(v¹ᵢ * v¹ᵢ) * J
            g22 = sum(v[2,:].^2) * jac          # g22 = sum(v²ᵢ * v²ᵢ) * J  
            g33 = v[3,3] * v[3,3] * jac         # g33 = v³₃ * v³₃ * J
            g23 = v[2,3] * v[3,3] * jac         # g23 = v²₃ * v³₃ * J
            g31 = v[3,3] * v[1,3] * jac         # g31 = v³₃ * v¹₃ * J
            g12 = sum(v[1,:] .* v[2,:]) * jac   # g12 = sum(v¹ᵢ * v²ᵢ) * J
            
            # Store results (convert to 1-based indexing for Julia)
            # Fortran: metric%fs(ipsi,itheta,1:8) = [g11,g22,g33,g23,g31,g12,jac,jac1]
            metric.fs[ipsi+1, itheta+1, 1] = g11
            metric.fs[ipsi+1, itheta+1, 2] = g22  
            metric.fs[ipsi+1, itheta+1, 3] = g33
            metric.fs[ipsi+1, itheta+1, 4] = g23
            metric.fs[ipsi+1, itheta+1, 5] = g31
            metric.fs[ipsi+1, itheta+1, 6] = g12
            metric.fs[ipsi+1, itheta+1, 7] = jac
            metric.fs[ipsi+1, itheta+1, 8] = jac1
        end
    end
    
    println("✅ Grid computation complete.")
    println("🔧 Fitting Fourier-spline representation...")
    
    # Fit the computed data to Fourier-spline representation
    # Fortran equivalent: CALL fspline_setup(metric, ...)
    try
        # Set up parameters for fitting
        fit_method = fft_flag ? 2 : 1  # 2=FFT, 1=integral
        bctype = 2  # Periodic boundary conditions for theta
        
        # Use JPEC's fspline_setup function to create the fitted representation
        # Julia equivalent of: CALL fspline_setup(xs, ys, fs, mband, bctype, fit_method)
        metric.fspline = JPEC.SplinesMod.fspline_setup(
            metric.xs, 
            metric.ys, 
            metric.fs, 
            mband, 
            bctype=bctype, 
            fit_method=fit_method,
            fit_flag=true
        )
        
        println("✅ Fourier-spline fitting successful.")
        
    catch e
        println("⚠️  Fourier-spline fitting failed: $e")
        println("   Proceeding with grid data only...")
        metric.fspline = nothing
    end
    
    # Print summary statistics
    println("\n📈 Metric Tensor Summary:")
    for i in 1:8
        component_data = metric.fs[:, :, i]
        min_val = minimum(component_data)
        max_val = maximum(component_data)
        println("   $(metric.title[i]): [$(round(min_val, digits=6)), $(round(max_val, digits=6))]")
    end
    
    println("🎉 Metric tensor calculation complete!\n")
    
    return metric
end

"""
    fourfit_make_matrix(metric::MetricData, sq; 
                       nn=1, mlow=-5, mhigh=5, 
                       power_flag=false, feval_flag=false, sas_flag=false, psilim=1.0)

Main function to construct MHD coefficient matrices and fit them to cubic splines.
This is a Julia port of the Fortran subroutine fourfit_make_matrix.

Arguments:
- metric: MetricData object containing fitted metric tensor components
- sq: JPEC 1D spline object containing equilibrium profiles  
- nn: Toroidal mode number (default: 1)
- mlow, mhigh: Range of poloidal mode numbers (default: -5 to 5)
- power_flag: Apply power corrections (default: false)
- feval_flag: Compute eigenvalues (default: false)
- sas_flag: Enable boundary analysis (default: false)
- psilim: Boundary flux surface (default: 1.0)

Returns:
- matrix_data: MatrixData object containing computed coefficient matrices

Fortran equivalent:
```fortran
SUBROUTINE fourfit_make_matrix
```
"""
function fourfit_make_matrix(metric::MetricData, sq; 
                           nn::Int=1, 
                           mlow::Int=-5, 
                           mhigh::Int=5,
                           power_flag::Bool=false,
                           feval_flag::Bool=false,
                           sas_flag::Bool=false,
                           psilim::Float64=1.0)
    
    println("🔧 Starting MHD matrix calculation...")
    
    # Constants and parameters
    twopi = 2π
    ifac = im  # Julia's imaginary unit
    
    # Derived parameters
    mpert = mhigh - mlow + 1  # Number of perturbed modes
    mpsi = metric.mpsi
    mband = metric.mband
    
    println("   Mode range: $mlow to $mhigh ($(mpert) modes)")
    println("   Toroidal mode: n = $nn")
    println("   Bandwidth: $mband")
    
    # Initialize matrix data structure
    matrix_data = MatrixData(mpsi, mpert, mband, mlow, mhigh, nn)
    matrix_data.xs .= metric.xs
    
    # Allocate coefficient matrices
    amat = zeros(ComplexF64, mpert, mpert)
    bmat = zeros(ComplexF64, mpert, mpert)
    cmat = zeros(ComplexF64, mpert, mpert)
    ipiva = zeros(Int, mpert)
    
    # Storage for coefficient matrices (simplified - would use JPEC's cspline in full implementation)
    fmat_storage = zeros(ComplexF64, mpsi+1, mpert, mpert)
    gmat_storage = zeros(ComplexF64, mpsi+1, mpert, mpert)
    kmat_storage = zeros(ComplexF64, mpsi+1, mpert, mpert)
    
    # Storage for Fourier coefficients (use Dict for negative indices)
    g11 = Dict{Int, ComplexF64}()
    g22 = Dict{Int, ComplexF64}()
    g33 = Dict{Int, ComplexF64}()
    g23 = Dict{Int, ComplexF64}()
    g31 = Dict{Int, ComplexF64}()
    g12 = Dict{Int, ComplexF64}()
    jmat = Dict{Int, ComplexF64}()
    jmat1 = Dict{Int, ComplexF64}()
    
    # Initialize dictionaries with zeros for all modes
    for dm in -mband:mband
        g11[dm] = 0.0 + 0.0im
        g22[dm] = 0.0 + 0.0im
        g33[dm] = 0.0 + 0.0im
        g23[dm] = 0.0 + 0.0im
        g31[dm] = 0.0 + 0.0im
        g12[dm] = 0.0 + 0.0im
        jmat[dm] = 0.0 + 0.0im
        jmat1[dm] = 0.0 + 0.0im
    end
    
    # Identity matrix in Fourier space
    imat = Dict{Int, ComplexF64}()
    for dm in -mband:mband
        imat[dm] = 0.0 + 0.0im
    end
    imat[0] = 1.0
    
    println("📊 Computing coefficient matrices on grid...")
    
    # Main loop over flux surfaces
    for ipsi in 0:mpsi
        psi_idx = ipsi + 1  # Convert to Julia 1-based indexing
        
        # Get equilibrium quantities at this flux surface
        # Note: This is simplified - would use proper spline evaluation
        psifac = matrix_data.xs[psi_idx]
        
        # Evaluate 1D profiles (F, P, Phi, q) at this psi
        # For now, use simplified evaluation - in full version would use JPEC.SplinesMod.spline_eval
        profiles = zeros(4)
        if psifac >= 0.01 && psifac <= 1.0
            # Simplified profile evaluation - replace with proper spline evaluation
            profiles[1] = 1.0     # F (toroidal field function)
            profiles[2] = 0.1     # P (pressure)  
            profiles[3] = psifac  # Phi (toroidal flux)
            profiles[4] = 1.5     # q (safety factor)
        end
        
        p1 = 0.0      # dP/dpsi (would be computed from spline derivative)
        q = profiles[4]
        q1 = 0.1      # dq/dpsi (would be computed from spline derivative)
        jtheta = -1.0 # -dF/dpsi (would be computed from spline derivative)
        
        chi1 = twopi * 1.0  # Simplified - would use psio from equilibrium
        nq = nn * q
        
        # Extract Fourier coefficients from fitted metric tensor
        if metric.fspline !== nothing
            # In full implementation, would extract Fourier coefficients from metric.fspline
            # For now, use simplified approach
            try
                # This is a placeholder - actual implementation would extract
                # Fourier coefficients from the fitted spline
                for dm in -mband:mband
                    g11[dm] = 1.0 + 0.1 * dm  # Simplified
                    g22[dm] = 2.0 + 0.05 * dm
                    g33[dm] = 1.5 + 0.02 * dm
                    g23[dm] = 0.1 * dm
                    g31[dm] = 0.05 * dm
                    g12[dm] = 0.02 * dm
                    jmat[dm] = 1.0
                    jmat1[dm] = 0.1
                end
                
                # Apply conjugate symmetry for positive modes
                for dm in 1:mband
                    g11[dm] = conj(g11[-dm])
                    g22[dm] = conj(g22[-dm])
                    g33[dm] = conj(g33[-dm])
                    g23[dm] = conj(g23[-dm])
                    g31[dm] = conj(g31[-dm])
                    g12[dm] = conj(g12[-dm])
                    jmat[dm] = conj(jmat[-dm])
                    jmat1[dm] = conj(jmat1[-dm])
                end
                
            catch e
                println("⚠️  Fourier coefficient extraction failed: $e")
                # Use default values
                for dm in -mband:mband
                    g11[dm] = 1.0 + 0.0im
                    g22[dm] = 2.0 + 0.0im
                    g33[dm] = 1.5 + 0.0im
                end
            end
        else
            # Use default values if no fitted spline available
            println("⚠️  No fitted metric spline, using default values")
            for dm in -mband:mband
                g11[dm] = 1.0 + 0.0im
                g22[dm] = 2.0 + 0.0im
                g33[dm] = 1.5 + 0.0im
            end
        end
        
        # Zero out matrices for this flux surface
        fill!(amat, 0.0)
        fill!(bmat, 0.0)
        fill!(cmat, 0.0)
        
        # Construct coefficient matrices
        # Loop over perturbed Fourier components
        ipert = 0
        for m1 in mlow:mhigh
            ipert += 1
            singfac1 = m1 - nq
            
            for dm in max(1-ipert, -mband):min(mpert-ipert, mband)
                m2 = m1 + dm
                singfac2 = m2 - nq
                jpert = ipert + dm
                
                if jpert >= 1 && jpert <= mpert
                    # Construct primitive matrices (Fortran equations translated)
                    amat[ipert, jpert] = twopi^2 * (nn^2 * g22[dm] + 
                                       nn * (m1 + m2) * g23[dm] + 
                                       m1 * m2 * g33[dm])
                    
                    bmat[ipert, jpert] = -twopi * ifac * chi1 * 
                                       (nn * g22[dm] + (m1 + nq) * g23[dm] + 
                                        m1 * q * g33[dm])
                    
                    cmat[ipert, jpert] = twopi * ifac * (
                        twopi * ifac * chi1 * singfac2 * (nn * g12[dm] + m1 * g31[dm]) -
                        q1 * chi1 * (nn * g23[dm] + m1 * g33[dm])) -
                        twopi * ifac * (jtheta * singfac1 * imat[dm] + 
                                      nn * p1 / chi1 * jmat[dm])
                end
            end
        end
        
        # Store matrices (simplified storage)
        fmat_storage[psi_idx, :, :] = amat  # In full version, would compute F, G, K matrices
        gmat_storage[psi_idx, :, :] = bmat
        kmat_storage[psi_idx, :, :] = cmat
        
        # Progress indicator
        if ipsi % 10 == 0 || ipsi == mpsi
            println("   Processed flux surface $ipsi/$mpsi (ψ=$(round(psifac, digits=3)))")
        end
    end
    
    println("✅ Matrix computation complete.")
    
    # Store results in matrix_data
    matrix_data.amat = amat
    matrix_data.bmat = bmat
    matrix_data.cmat = cmat
    matrix_data.ipiva = ipiva
    
    println("🎉 MHD matrix calculation complete!\n")
    
    return matrix_data
end

"""
    compute_eigenvalues(matrix_data::MatrixData, psi::Float64)

Compute eigenvalues of the MHD matrix at a specific flux surface.
This is a simplified version of the eigenvalue computation.
"""
function compute_eigenvalues(matrix_data::MatrixData, psi::Float64)
    try
        # This would evaluate the splined matrices at the given psi
        # For now, use the stored matrices
        if matrix_data.amat !== nothing
            eigenvals = eigvals(matrix_data.amat)
            return eigenvals
        else
            println("⚠️  No matrix data available")
            return ComplexF64[]
        end
    catch e
        println("❌ Eigenvalue computation failed: $e")
        return ComplexF64[]
    end
end

"""
    fspline_eval_metric(metric::MetricData, psi::Float64, theta::Float64)

Evaluate the fitted metric tensor at a specific (psi, theta) point.
If no fitted spline is available, uses linear interpolation on the grid data.

Fortran equivalent:
```fortran
CALL fspline_eval(metric, [psi], [theta], f_result, 0)
```
"""
function fspline_eval_metric(metric::MetricData, psi::Float64, theta::Float64)
    if metric.fspline !== nothing
        # Use the fitted Fourier-spline for evaluation
        # Julia equivalent of: CALL fspline_eval(metric, [psi], [theta], f_result, 0)
        try
            f_result = JPEC.SplinesMod.fspline_eval(metric.fspline, [psi], [theta], 0)
            return f_result[1, 1, :]  # Return all 8 components
        catch e
            println("⚠️  Spline evaluation failed: $e, falling back to grid interpolation")
        end
    end
    
    # Fallback: Simple linear interpolation on grid data
    # This is a simplified implementation - could be improved with proper interpolation
    psi_idx = searchsortedfirst(metric.xs, psi)
    theta_idx = searchsortedfirst(metric.ys, theta)
    
    # Clamp indices to valid range
    psi_idx = clamp(psi_idx, 2, length(metric.xs))
    theta_idx = clamp(theta_idx, 2, length(metric.ys))
    
    # Simple nearest-neighbor for now (could be improved)
    return metric.fs[psi_idx, theta_idx, :]
end

"""
    plot_metric_components(metric::MetricData; component_indices=[1,2,3,7])

Create plots of selected metric tensor components.
Requires Plots.jl to be loaded externally before calling this function.

Fortran equivalent: Visualization routines for metric tensor data
"""
function plot_metric_components(metric::MetricData; component_indices=[1,2,3,7])
    # Check if Plots is available
    if !isdefined(Main, :Plots)
        println("⚠️  Plots.jl not loaded. Please run 'using Plots' first.")
        return nothing
    end
    
    try
        plots = []
        for idx in component_indices
            if 1 <= idx <= 8
                p = Main.Plots.heatmap(
                    metric.xs, 
                    metric.ys, 
                    metric.fs[:, :, idx]',
                    title="$(metric.title[idx])",
                    xlabel="ψ",
                    ylabel="θ", 
                    aspect_ratio=:equal
                )
                push!(plots, p)
            end
        end
        
        return Main.Plots.plot(plots..., layout=(2,2), size=(800, 600))
        
    catch e
        println("⚠️  Plotting failed: $e")
        println("   Make sure Plots.jl is loaded: using Plots")
        return nothing
    end
end

end # module FourfitMetric

Main.FourfitMetric

### Metric example

In [27]:
# 간단한 Fourfit Metric 사용 예시
println("📚 간단한 사용 예시")
println("="^30)

# 이미 로드된 plasma_eq를 사용해서 metric 계산
using .FourfitMetric

println("1️⃣  기본 metric 계산...")
try
    # 기본 매개변수로 metric 계산
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D 기하 스플라인
        plasma_eq.sq;     # 1D 프로파일 스플라인
        mpsi=30,          # 반지름 방향 격자점
        mtheta=64,        # 극각 방향 격자점  
        mband=8,          # 푸리에 대역폭
        fft_flag=true     # FFT 사용
    )
    
    println("✅ Metric 계산 완료!")
    
    # 결과 확인
    println("\n2️⃣  결과 확인...")
    println("   격자 크기: $(size(metric_result.fs))")
    println("   성분 개수: $(length(metric_result.title))")
    
    # 각 성분의 범위 출력
    for i in 1:8
        component = metric_result.fs[:, :, i]
        min_val = minimum(component)
        max_val = maximum(component)
        println("   $(metric_result.title[i]): [$(round(min_val, digits=4)), $(round(max_val, digits=4))]")
    end
    
    # 특정 점에서 평가 (스플라인이 있는 경우)
    if metric_result.fspline !== nothing
        println("\n3️⃣  특정 점에서 평가...")
        psi_test = 0.5
        theta_test = π/2
        
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   (ψ=$psi_test, θ=$(round(theta_test/π, digits=2))π)에서:")
        println("   g11 = $(round(components[1], digits=6))")
        println("   g22 = $(round(components[2], digits=6))")
        println("   jac = $(round(components[7], digits=6))")
    end
    
    println("\n✅ 간단한 예시 완료!")
    
catch e
    println("❌ 에러 발생: $e")
end

📚 간단한 사용 예시
1️⃣  기본 metric 계산...
🔧 Starting metric tensor calculation...


   Grid: 31 × 65, mband: 8
📊 Computing metric tensor components on grid...
✅ Grid computation complete.
🔧 Fitting Fourier-spline representation...
✅ Fourier-spline fitting successful.

📈 Metric Tensor Summary:
   g11: [0.022144, 3.9425346241307e7]
   g22: [0.234775, 2.5746938041326e7]
   g33: [0.0, 12.199776]
   g23: [-25.110338, 32.165598]
   g31: [-25.626758, 20.393748]
   g12: [-5.692796494019e6, 1.328203406148e6]
   jmat: [15.829441, 205.419335]
   jmat1: [-1886.708238, 2185.874666]
🎉 Metric tensor calculation complete!

✅ Metric 계산 완료!

2️⃣  결과 확인...
   격자 크기: (31, 65, 8)
   성분 개수: 8
   g11: [0.0221, 3.94253462413e7]
   g22: [0.2348, 2.57469380413e7]
   g33: [0.0, 12.1998]
   g23: [-25.1103, 32.1656]
   g31: [-25.6268, 20.3937]
   g12: [-5.692796494e6, 1.3282034061e6]
   jmat: [15.8294, 205.4193]
   jmat1: [-1886.7082, 2185.8747]

3️⃣  특정 점에서 평가...
   (ψ=0.5, θ=0.5π)에서:
   g11 = 1554.145306
   g22 = 13287.812608
   jac = 27.983169

✅ 간단한 예시 완료!
🔧 Fitting Fourier-spline representat

### Bicube eval debug

In [29]:
# bicube_eval 함수 디버깅
println("🔍 bicube_eval 반환값 구조 디버깅")
println("="^40)

if @isdefined(plasma_eq)
    try
        # 테스트 좌표
        psi_test = 0.5
        theta_test = π/2
        
        println("Testing bicube_eval at psi=$psi_test, theta=$theta_test")
        
        # bicube_eval 호출
        f, fx, fy = JPEC.SplinesMod.bicube_eval(plasma_eq.rzphi, psi_test, theta_test, 1)
        
        println("반환값 구조:")
        println("  f: $(typeof(f)), size: $(size(f))")
        println("  fx: $(typeof(fx)), size: $(size(fx))")  
        println("  fy: $(typeof(fy)), size: $(size(fy))")
        
        println("f 내용: $f")
        println("fx 내용: $fx")
        println("fy 내용: $fy")
        
        # 개별 요소 접근 테스트
        println("\n개별 요소 접근 테스트:")
        try
            println("f[1] = $(f[1])")
            println("f[2] = $(f[2])")
            println("f[3] = $(f[3])")
            println("f[4] = $(f[4])")
        catch e
            println("f 요소 접근 실패: $e")
        end
        
        try
            println("fx[1] = $(fx[1])")
            println("fx[4] = $(fx[4])")
        catch e
            println("fx 요소 접근 실패: $e")
        end
        
    catch e
        println("❌ 디버깅 실패: $e")
        println("스택 트레이스:")
        println(stacktrace(catch_backtrace()))
    end
else
    println("❌ plasma_eq가 정의되지 않았습니다.")
end

🔍 bicube_eval 반환값 구조 디버깅
Testing bicube_eval at psi=0.5, theta=1.5707963267948966
반환값 구조:
  f: Vector{Float64}, size: (4,)
  fx: Vector{Float64}, size: (4,)
  fy: Vector{Float64}, size: (4,)
f 내용: [1.2069187318112993, 1.2968477783799697, 0.03060017828247886, 19.066900473973202]
fx 내용: [2.873662517431406, 1.0215974711532225, 0.1918592948635952, 3.670999794526376]
fy 내용: [4.5450437186906765, 1.2194764633845154, 1.3018145632230462, 95.07941580312034]

개별 요소 접근 테스트:
f[1] = 1.2069187318112993
f[2] = 1.2968477783799697
f[3] = 0.03060017828247886
f[4] = 19.066900473973202
fx[1] = 2.873662517431406
fx[4] = 3.670999794526376


### Matrix eval

In [30]:
# Fourfit Matrix 계산 사용 예시
println("🚀 Fourfit Matrix 계산 예시")
println("="^40)

if @isdefined(plasma_eq)
    using .FourfitMetric
    
    try
        # 1단계: Metric tensor 계산
        println("1️⃣  Metric tensor 계산...")
        metric_result = FourfitMetric.fourfit_make_metric(
            plasma_eq.rzphi,
            plasma_eq.sq;
            mpsi=20,
            mtheta=32,
            mband=3,
            fft_flag=false
        )
        
        println("✅ Metric tensor 계산 완료!")
        
        # 2단계: Matrix 계산
        println("\n2️⃣  MHD coefficient matrix 계산...")
        matrix_result = FourfitMetric.fourfit_make_matrix(
            metric_result,
            plasma_eq.sq;
            nn=1,           # 토로이달 모드 번호
            mlow=-3,        # 최소 폴로이달 모드
            mhigh=3,        # 최대 폴로이달 모드
            power_flag=false,
            feval_flag=true,
            sas_flag=false
        )
        
        println("✅ Matrix 계산 완료!")
        
        # 3단계: 결과 분석
        println("\n3️⃣  결과 분석...")
        println("   Matrix dimensions: $(size(matrix_result.amat))")
        println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
        println("   Number of modes: $(matrix_result.mpert)")
        println("   Bandwidth: $(matrix_result.mband)")
        
        # 고유값 계산 예시
        println("\n4️⃣  고유값 계산 예시...")
        if matrix_result.amat !== nothing
            eigenvals = FourfitMetric.compute_eigenvalues(matrix_result, 0.5)
            if length(eigenvals) > 0
                println("   처음 몇 개 고유값:")
                for (i, λ) in enumerate(eigenvals[1:min(3, length(eigenvals))])
                    println("     λ[$i] = $(round(real(λ), digits=6)) + $(round(imag(λ), digits=6))i")
                end
            end
        end
        
        println("\n✅ 전체 예시 완료!")
        
    catch e
        println("❌ 에러 발생: $e")
        println("스택 트레이스:")
        println(stacktrace(catch_backtrace()))
    end
else
    println("❌ plasma_eq가 정의되지 않았습니다.")
    println("   먼저 3번째 셀을 실행해서 평형을 로드하세요.")
end

🚀 Fourfit Matrix 계산 예시
1️⃣  Metric tensor 계산...
🔧 Starting metric tensor calculation...
   Grid: 21 × 33, mband: 3
📊 Computing metric tensor components on grid...
✅ Grid computation complete.
🔧 Fitting Fourier-spline representation...
✅ Fourier-spline fitting successful.

📈 Metric Tensor Summary:
   g11: [0.022144, 3.9425346241307e7]
   g22: [0.259067, 729283.417328]
   g33: [0.0, 13.365317]
   g23: [-25.110338, 30.305035]
   g31: [-12.745112, 20.227383]
   g12: [-523589.094982, 408481.962479]
   jmat: [15.826482, 200.804638]
   jmat1: [-1886.708238, 2185.874666]
🎉 Metric tensor calculation complete!

✅ Metric tensor 계산 완료!

2️⃣  MHD coefficient matrix 계산...
🔧 Starting MHD matrix calculation...
   Mode range: -3 to 3 (7 modes)
   Toroidal mode: n = 1
   Bandwidth: 3
📊 Computing coefficient matrices on grid...
   Processed flux surface 0/20 (ψ=0.01)
   Processed flux surface 10/20 (ψ=0.505)
   Processed flux surface 20/20 (ψ=1.0)
✅ Matrix computation complete.
🎉 MHD matrix calculation c